## Pitzer College FYS Placement Program

__Authors: Brian Simpkins and Professor Sarah Gilman__

__Written in the fall of 2022__

In [ ]:
# First let's set up some imports
import numpy as np
import pandas as pd